# Training a neural network on MNIST with Keras

This simple example demonstrates how to plug TensorFlow Datasets (TFDS) into a Keras model.


Copyright 2020 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/datasets/keras_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/datasets/blob/master/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/datasets/docs/keras_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In [10]:
import warnings
warnings.filterwarnings('ignore')


In [11]:
import tensorflow as tf
import tensorflow_datasets as tfds

## Step 1: Create your input pipeline

Start by building an efficient input pipeline using advices from:
* The [Performance tips](https://www.tensorflow.org/datasets/performances) guide
* The [Better performance with the `tf.data` API](https://www.tensorflow.org/guide/data_performance#optimize_performance) guide


### Load a dataset

Load the MNIST dataset with the following arguments:

* `shuffle_files=True`: The MNIST data is only stored in a single file, but for larger datasets with multiple files on disk, it's good practice to shuffle them when training.
* `as_supervised=True`: Returns a tuple `(img, label)` instead of a dictionary `{'image': img, 'label': label}`.

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

I0000 00:00:1745426968.678887   59431 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745426968.765700   59431 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745426968.765842   59431 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745426968.767493   59431 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Shuffling /home/class/tensorflow_datasets/mnist/incomplete.00EYR1_3.0.1/mnist-train.tfrecord*...:   0%|       …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /home/class/tensorflow_datasets/mnist/incomplete.00EYR1_3.0.1/mnist-test.tfrecord*...:   0%|        …

Dataset mnist downloaded and prepared to /home/class/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [6]:
ds_test


<_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

### Build a training pipeline

Apply the following transformations:

* `tf.data.Dataset.map`: TFDS provide images of type `tf.uint8`, while the model expects `tf.float32`. Therefore, you need to normalize images.
* `tf.data.Dataset.cache` As you fit the dataset in memory, cache it before shuffling for a better performance.<br/>
__Note:__ Random transformations should be applied after caching.
* `tf.data.Dataset.shuffle`: For true randomness, set the shuffle buffer to the full dataset size.<br/>
__Note:__ For large datasets that can't fit in memory, use `buffer_size=1000` if your system allows it.
* `tf.data.Dataset.batch`: Batch elements of the dataset after shuffling to get unique batches at each epoch.
* `tf.data.Dataset.prefetch`: It is good practice to end the pipeline by prefetching [for performance](https://www.tensorflow.org/guide/data_performance#prefetching).

In [7]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

### Build an evaluation pipeline

Your testing pipeline is similar to the training pipeline with small differences:

 * You don't need to call `tf.data.Dataset.shuffle`.
 * Caching is done after batching because batches can be the same between epochs.

In [8]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Step 2: Create and train the model

Plug the TFDS input pipeline into a simple Keras model, compile the model, and train it.

In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


I0000 00:00:1745427263.831949   59516 service.cc:146] XLA service 0x751044005e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745427263.831967   59516 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2025-04-23 18:54:23.854439: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-23 18:54:23.905484: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


154/469 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9590 - sparse_categorical_accuracy: 0.7283

I0000 00:00:1745427266.370079   59516 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


469/469 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.6069 - sparse_categorical_accuracy: 0.8294 - val_loss: 0.1969 - val_sparse_categorical_accuracy: 0.9447
Epoch 2/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1821 - sparse_categorical_accuracy: 0.9494 - val_loss: 0.1378 - val_sparse_categorical_accuracy: 0.9618
Epoch 3/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1256 - sparse_categorical_accuracy: 0.9646 - val_loss: 0.1088 - val_sparse_categorical_accuracy: 0.9662
Epoch 4/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 979us/step - loss: 0.0951 - sparse_categorical_accuracy: 0.9724 - val_loss: 0.0978 - val_sparse_categorical_accuracy: 0.9694
Epoch 5/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0734 - sparse_categorical_accuracy: 0.9786 - val_loss: 0.0841 - val_sparse_categorical_accuracy: 0.9738
Epoch 6/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0598 - sparse_categorical_accuracy: 0.9834 - val_loss: 0.0797 - val_sparse_categorical_accuracy: 0.9750


...

397/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0615 - sparse_categorical_accuracy: 0.9834

420/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0615 - sparse_categorical_accuracy: 0.9833

444/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0615 - sparse_categorical_accuracy: 0.9833

468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0615 - sparse_categorical_accuracy: 0.9833

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0615 - sparse_categorical_accuracy: 0.9832 - val_loss: 0.0854 - val_sparse_categorical_accuracy: 0.9729
